In [52]:
import os
import operator
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
lbph_bin = [-1,0, 1, 2, 3, 4, 6, 7, 8, 12, 14, 15, 16, 24, 28, 30, 31, 32, 48, 56, 60, 62, 63, 64, 96, 112, 120, 124, 126, 127, 128, 129, 131, 135, 143, 159, 191, 192, 193, 195, 199, 207, 223, 224, 225, 227, 231, 239, 240, 241, 243, 247, 248, 249, 251, 252, 253, 254,255]
bins_number = 59
height =245
width = 210
lbph_length = 2478
chisquare_weight = [[0,1,1,1,1,0],
                    [2,2,1,1,2,2],
                    [2,4,4,4,4,2],
                    [0,1,0,0,1,0],
                    [1,1,2,2,1,1],
                    [1,2,4,4,2,1],
                    [0,1,2,2,1,0],
                   ]
labels = ['anger', 'disgust', 'fear', 'joy','neutral', 'sadness', 'surprise']

In [53]:
def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

In [54]:
def lbp_calculated_pixel(img, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val    

In [55]:
def calculate_lphp_of_image(image):
    im_hist = []
    for i in range(0,height,35):
        for j in range(0,width,35):
            img = image[i:(i+35),j:(j+35)]
            reg_hist = [0] * bins_number
            for m in range (0,35):
                for n in range(0,35):
                    tempt = lbp_calculated_pixel(img, m, n)
                    if lbph_bin.count(tempt) > 0:
                        reg_hist[lbph_bin.index(tempt)] = reg_hist[lbph_bin.index(tempt)] + 1
                    else:
                        reg_hist[0] = reg_hist[0] + 1 
            im_hist.extend(reg_hist)
    return im_hist

In [50]:
def chisquare_2lbph(lbph1, lbph2):
    result = 0
    for i in range (0,7):
        for j in range(0,6):
            if chisquare_weight[i][j] != 0:
                s = lbph1[i*210+(j-1)*35:i*210+j*35]
                m = lbph2[i*210+(j-1)*35:i*210+j*35]
                num = np.subtract(s,m)
                num = np.power(num,2)
                den = np.add(s,m)
                den = np.where(den == 0, 1, den)
        result = result + np.sum(chisquare_weight[i][j] * (np.divide(num,den)))
    return result

In [56]:
def get_list_lbph_tag():
    entries = os.listdir('data/')
    result = []
    tags = []
    print(entries)
    i = 0
    for entry in entries:
        for image_path in os.listdir('data/'+entry+"/"):
            img_bgr = cv2.imread('data/'+ entry +'/'+image_path)
            img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
            height,width = img_gray.shape
            result.append(calculate_lphp_of_image(img_gray))
            tags.append(i)
        i = i + 1
    return result,tags

In [57]:
lbph_list, tags = get_list_lbph_tag()

['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']


In [58]:
tags_list = []
lbphs_list = []
for i in range (0, 10):
    j = i * 6
    tempt = []
    tempt1 = []
    for m in range(0,7):
        tempt.extend(tags[j:j+6])
        tempt1.extend(lbph_list[j:j+6])
        j = j + 60
    tags_list.append(tempt)
    lbphs_list.append(tempt1)

[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 

In [62]:
kf = KFold(n_splits=10)
total_precision = 0
for train_index, test_index in kf.split(tags_list):
    print("TRAIN:", train_index, "TEST:", test_index)
    lbphs_train = [[0]*lbph_length]*7
    lbphs_test = lbphs_list[test_index[0]]
    tag_test = tags_list[test_index[0]]
    precision = 0
    for i in train_index:
        for j in range(0,7):
            for m in range(j*6,j*6+6):
                lbphs_train[j] = np.add(lbphs_train[j],lbphs_list[i][m])
    for i in range(0,7):
        lbphs_train[i] = np.divide(lbphs_train[i],54)
    for i in range(0,42):
        result = []
        for j in range(0,7):
            result.append(chisquare_2lbph(lbphs_test[i],lbphs_train[j]))
        if result.index(min(result)) == tag_test[i]:
            precision = precision + 1
    total_precision = total_precision + precision/42
    print("PRECISION:",precision/42)
print("  ")
print("AVERAGE PRECISION:",total_precision/10)

TRAIN: [1 2 3 4 5 6 7 8 9] TEST: [0]
PRECISION: 0.3333333333333333
TRAIN: [0 2 3 4 5 6 7 8 9] TEST: [1]
PRECISION: 0.2619047619047619
TRAIN: [0 1 3 4 5 6 7 8 9] TEST: [2]
PRECISION: 0.21428571428571427
TRAIN: [0 1 2 4 5 6 7 8 9] TEST: [3]
PRECISION: 0.47619047619047616
TRAIN: [0 1 2 3 5 6 7 8 9] TEST: [4]
PRECISION: 0.19047619047619047
TRAIN: [0 1 2 3 4 6 7 8 9] TEST: [5]
PRECISION: 0.2857142857142857
TRAIN: [0 1 2 3 4 5 7 8 9] TEST: [6]
PRECISION: 0.35714285714285715
TRAIN: [0 1 2 3 4 5 6 8 9] TEST: [7]
PRECISION: 0.23809523809523808
TRAIN: [0 1 2 3 4 5 6 7 9] TEST: [8]
PRECISION: 0.38095238095238093
TRAIN: [0 1 2 3 4 5 6 7 8] TEST: [9]
PRECISION: 0.2857142857142857
  
AVERAGE PRECISION: 0.30238095238095236
